# Preparation
## Downloading necessary packages

In [56]:
using JuDGE, JuMP, GLPK, CSV, DataFrames, Chain, Statistics

## Reading csv files

In [77]:

data = CSV.read("degree_hours.csv", DataFrame)
periods = unique(data[!,"period"])
paths =   unique(data[!,"ssp"])

rows = []

period = periods[1]
ssp = '-'

# Using Chain for more readable pipeline
result = @chain data begin
    filter(:period => ==(period), _)  # filter first period
    groupby([:quantile,:variable])
    combine(:count => mean)
end

push!(rows,Dict(
    :period => period,
    :ssp => "-",
    :degree_hours => result
))

for period in periods[2:end]
    for ssp in paths
        # Using Chain for more readable pipeline
        result = @chain data begin
            filter(:period => ==(period), _) 
            filter(:ssp => ==(ssp), _) 
            select(:quantile, :variable, :count)
        end 
        push!(rows,Dict(
            :period => period,
            :ssp => ssp,
            :degree_hours => result
        ))
    end
end
println("ok")


ok


In [45]:

mytree = narytree(2,2)
JuDGE.print_tree(mytree)

--1
  --11
    --111
    --112
  --12
    --121
    --122


In [3]:
invest_cost = Dict( zip( collect(mytree,order=:breadth), [15, 8, 8, 4, 4, 4, 4]) )


item_volume = Dict( zip( collect(mytree,order=:breadth), [ [4, 3, 3, 1, 2],
                                                           [5, 3, 4, 2, 1],
                                                           [5, 4, 2, 7, 2],
                                                           [5, 4, 1, 8, 2],
                                                           [3, 1, 5, 6, 3],
                                                           [2, 5, 8, 4, 6],
                                                           [7, 5, 4, 2, 3] ]) )

item_reward = Dict( zip( collect(mytree,order=:breadth), [ [32, 9, 9, 4, 8],
                                                           [30, 12, 40, 10, 9],
                                                           [20, 28, 12, 42, 12],
                                                           [40, 28, 9, 24, 10],
                                                           [15, 7, 20, 48, 12],
                                                           [10, 30, 54, 32, 30],
                                                           [32, 25, 24, 14, 24] ]) )

Dict{AbstractTree, Vector{Int64}} with 7 entries:
  Subtree rooted at node 12 containing 3 nodes => [20, 28, 12, 42, 12]
  Leaf node 112                                => [15, 7, 20, 48, 12]
  Subtree rooted at node 1 containing 7 nodes  => [32, 9, 9, 4, 8]
  Leaf node 121                                => [10, 30, 54, 32, 30]
  Leaf node 122                                => [32, 25, 24, 14, 24]
  Leaf node 111                                => [40, 28, 9, 24, 10]
  Subtree rooted at node 11 containing 3 nodes => [30, 12, 40, 10, 9]

In [4]:

JuDGE.print_tree(mytree,item_reward)

--1 ([32, 9, 9, 4, 8])
  --11 ([30, 12, 40, 10, 9])
    --111 ([40, 28, 9, 24, 10])
    --112 ([15, 7, 20, 48, 12])
  --12 ([20, 28, 12, 42, 12])
    --121 ([10, 30, 54, 32, 30])
    --122 ([32, 25, 24, 14, 24])


In [5]:
num_items = 5
num_invest = 6
initial_volume = 6
invest_volume = [2,2,2,3,3,3]

6-element Vector{Int64}:
 2
 2
 2
 3
 3
 3

In [20]:
JuDGE_SP_Solver = optimizer_with_attributes(GLPK.Optimizer, "msg_lev" => 0, "mip_gap" => 0.0)
function sub_problems(node)
   sp = Model(JuDGE_SP_Solver)

   @expansion(sp, invest[1:num_invest] <= 10, duration = 2)
   @capitalcosts(sp, sum(invest[i]*invest_volume[i] for i=1:num_invest)*invest_cost[node])

   @variable(sp, y[1:num_items], Bin)

   @constraint(sp, BagExtension, sum(y[i]*item_volume[node][i] for i in 1:num_items) <=
        initial_volume + sum(invest_volume[i] * invest[i] for i in 1:num_invest))

   @objective(sp, Min, sum(-item_reward[node][i] * y[i] for i in 1:num_items))

   return sp
end

sub_problems (generic function with 1 method)

In [21]:
JuDGE_MP_Solver = optimizer_with_attributes((method=GLPK.INTERIOR) -> GLPK.Optimizer(),
							"msg_lev" => 0, "mip_gap" => 0.0)
judy = JuDGEModel(mytree, ConditionallyUniformProbabilities, sub_problems, JuDGE_MP_Solver)


1

[ Info: Establishing JuDGE model for tree: Subtree rooted at node 1 containing 7 nodes
[ Info: Building JuMP Model for node 


1111111212121122s...Complete
Complete
Complete


[ Info: Checking sub-problem format...
[ Info: Building master problem...


JuDGE Model with:
  Tree: Subtree rooted at node 1 containing 7 nodes
  Expansion variables: invest 

In [22]:
JuDGE.solve(judy,termination=Termination(rlx_abstol=10^-7),verbose=1)


Relaxed ObjVal  |   Upper Bound   Lower Bound  |  Absolute Diff   Relative Diff  |  Fractional  |      Time     Iter
  2.850000e+03  |  2.850000e+03          -Inf  |            Inf             NaN  |           0  |     0.002        1
  1.484000e+03  |  1.484000e+03 -5.260000e+02  |   2.010000e+03    3.821293e+00  |           0  |     0.003        2
 -1.635000e+02  | -1.635000e+02 -5.260000e+02  |   3.625000e+02    6.891635e-01  |           0  |     0.003        3
 -1.840000e+02  | -1.840000e+02 -5.260000e+02  |   3.420000e+02    6.501901e-01  |           0  |     0.004        4
 -1.840000e+02  | -1.840000e+02 -5.260000e+02  |   3.420000e+02    6.501901e-01  |           0  |     0.004        5
 -1.840000e+02  | -1.840000e+02 -5.260000e+02  |   3.420000e+02    6.501901e-01  |           0  |     0.004        6
 -1.840000e+02  | -1.840000e+02 -5.260000e+02  |   3.420000e+02    6.501901e-01  |           0  |     0.005        7
 -1.915000e+02  | -1.915000e+02 -1.915000e+02  |  -1.136868e-13

[ Info: Solving JuDGE model for tree: Subtree rooted at node 1 containing 7 nodes
